# Import Packages

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

# Generate Noisy Training Data
- We will build a neural net (nn) to fit a quadratic function.  
- We will first generate some noisy training data to simulate real-world data.  
- NNs are noise-resistant.

In [ ]:
# Generate synthetic data
np.random.seed(42) # For reproducibility
x = np.linspace(-10, 10, 100000)
a, b, c = 2, 3, -1 # True parameters. We will try to recover these parameters
# True model + noise. 
y = a * x ** 2 + b * x + c + np.random.normal(0, 1, len(x)) 

print(x.shape, y.shape)

# Build Dataset and DataLoader

In [ ]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# split train, val, test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.4, random_state=42)


train_dataset = MyDataset(x_train, y_train)
val_dataset = MyDataset(x_val, y_val)
test_dataset = MyDataset(x_test, y_test)

BATCH_SIZE = 128
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Define Model
A neural network with only 3 learnable parameters.

In [ ]:
class NetQuadratic(nn.Module):
    def __init__(self):
        super(NetQuadratic, self).__init__()
        # learnable parameters
        self._a = nn.Parameter(torch.randn(1))
        self._b = nn.Parameter(torch.randn(1))
        self._c = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return self._a * x ** 2 + self._b * x + self._c

# Initialize Model, Loss, and Optimizer

In [ ]:
net = NetQuadratic()

# Loss and optimizer
criterion = nn.MSELoss()
LR=1e-3 # Learning rate
optimizer = optim.Adam(net.parameters(), lr=LR)

# Fit the Neural Network

In [ ]:
# Training loop
TOTAL_EPOCHS = 100
for epoch in range(TOTAL_EPOCHS):
    for i, (x_batch, y_batch) in enumerate(train_dataloader):
        optimizer.zero_grad()
        y_pred = net(x_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}/{TOTAL_EPOCHS}, Train Loss: {loss.item()}')

        # validation loss
        with torch.no_grad():
            avg_val_loss = 0
            for x_val_batch, y_val_batch in val_dataloader:
                y_val_pred = net(x_val_batch)
                val_loss = criterion(y_val_pred, y_val_batch)
                avg_val_loss += val_loss.item()
            avg_val_loss /= len(val_dataloader)
            print(f'Epoch {epoch}/{TOTAL_EPOCHS}, Val Loss: {avg_val_loss}')

# Check the Results

In [ ]:
net.eval()

# Test
with torch.no_grad():
    avg_test_loss = 0
    for x_test_batch, y_test_batch in test_dataloader:
        y_test_pred = net(x_test_batch)
        test_loss = criterion(y_test_pred, y_test_batch)
        avg_test_loss += test_loss.item()
    avg_test_loss /= len(test_dataloader)
    print(f'Test Loss: {avg_test_loss}')

# Print learned parameters
error = (net._a.item()-a)**2 + (net._b.item()-b)**2 + (net._c.item()-c)**2
print(f'Learned a: {net._a.item()}, b: {net._b.item()}, c: {net._c.item()}')
print(f'True a: {a}, b: {b}, c: {c}')
print("error: ", error)

if error < 1e-3:
    print("Success! Your model has converged to the true parameters.")
else:
    raise ValueError("Failed! The error is too large.")

# 

# Visualization

In [ ]:
# visualizing the training set
import matplotlib.pyplot as plt
# plot name
plt.title('Training set')
plt.scatter(x_train, y_train, s=1)

In [ ]:
# visualizing the learned model
x_tensor = torch.from_numpy(x).float()
y_pred = net(x_tensor)
y_pred = y_pred.detach().numpy()
plt.plot(x, y_pred, color='red')
plt.title('Learned model')
plt.show()